<a href="https://colab.research.google.com/github/UppuSushma/NLP-Lab/blob/main/NLP_Assignment_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use a simple dataset for English-to-French translation. You can either use a small dataset like this or download a more extensive dataset such as the Tab-delimited Bilingual Sentence Pairs dataset from Tatoeba or Parallel Corpus from the European Parliament.
# Example data (small English to French pairs)

data = [ ("hello", "bonjour"), ("how are you", "comment ça va"), ("I am fine", "je vais bien"), ("what is your name", "comment tu t'appelles"), ("my name is", "je m'appelle"), ("thank you", "merci"), ("goodbye", "au revoir") ]

**1 Data Preprocessing**

In [23]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample data
data = [("hello", "bonjour"), ("how are you", "comment ça va"), ("I am fine", "je vais bien"),
        ("what is your name", "comment tu t'appelles"), ("my name is", "je m'appelle"),
        ("thank you", "merci"), ("goodbye", "au revoir")]

# Split English and French sentences
english_texts, french_texts = zip(*data)

# Tokenize English text
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_texts)
eng_sequences = eng_tokenizer.texts_to_sequences(english_texts)
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_max_length = max(len(seq) for seq in eng_sequences)

# Tokenize French text
fr_tokenizer = Tokenizer()
fr_tokenizer.fit_on_texts(french_texts)
fr_sequences = fr_tokenizer.texts_to_sequences(french_texts)
fr_vocab_size = len(fr_tokenizer.word_index) + 1
fr_max_length = max(len(seq) for seq in fr_sequences)

# Pad sequences
eng_sequences = pad_sequences(eng_sequences, maxlen=eng_max_length, padding='post')
fr_sequences = pad_sequences(fr_sequences, maxlen=fr_max_length, padding='post')


**2 Build Seq2Seq Model**

In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Encoder
encoder_inputs = Input(shape=(eng_max_length,))
encoder_embedding = Embedding(eng_vocab_size, 256)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(256, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(fr_max_length,))
decoder_embedding = Embedding(fr_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(fr_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 4, 256)         │          3,840 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 3, 256)         │          3,584 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 256), (None,   │        525,312 │ embedding_2[0][0]      │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, 3, 256),       │        525,312 │ embedding_3[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm_2[0][1],          │
│                           │ 256)]                  │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 3, 14)          │          3,598 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,061,646 (4.05 MB)

 Trainable params: 1,061,646 (4.05 MB)

 Non-trainable params: 0 (0.00 B)

**3 Preparing the Data for Training**

In [25]:
# Shift decoder input sequences by one timestep to get target sequences
fr_target_sequences = np.expand_dims(fr_sequences, -1)

# Prepare training input and output
X_train = [eng_sequences, fr_sequences]
Y_train = fr_target_sequences


**4 Train the model on the dataset**

In [26]:
# Train the model
model.fit(X_train, Y_train, batch_size=2, epochs=100, validation_split=0.2)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 270ms/step - accuracy: 0.2167 - loss: 2.6322 - val_accuracy: 0.5000 - val_loss: 2.5919
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5167 - loss: 2.5568 - val_accuracy: 0.5000 - val_loss: 2.5111
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.4625 - loss: 2.4678 - val_accuracy: 0.5000 - val_loss: 2.4055
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.4917 - loss: 2.3409 - val_accuracy: 0.5000 - val_loss: 2.2565
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.3250 - loss: 2.2557 - val_accuracy: 0.5000 - val_loss: 2.0915
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.4292 - loss: 2.0092 - val_accuracy: 0.5000 - val_loss: 2.0085
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.4708 - loss: 1.7961 - val_accuracy: 0.5000 - val_loss: 2.2456
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5167 - loss: 1.8201 - val_accuracy: 0.5000 - val_loss

**5 Inference Setup for Translation**

In [27]:
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)


**6 Translate New Sentences**

In [28]:
def translate_sentence(input_text):
    # Tokenize and pad input text
    input_seq = eng_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=eng_max_length, padding='post')

    # Encode input
    states_value = encoder_model.predict(input_seq)

    # Initialize target sequence with start token
    target_seq = np.zeros((1, 1))
    translation = ''

    for _ in range(fr_max_length):
        # Predict the next word
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample token with highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = fr_tokenizer.index_word.get(sampled_token_index, None)

        # If end of sentence, break
        if sampled_word == 'end' or sampled_word is None:
            break
        translation += sampled_word + ' '

        # Update target sequence and states
        target_seq = np.array([[sampled_token_index]])
        states_value = [h, c]

    return translation.strip()


**7 Experimenting and Improving the Model by large dataset and hyper tune parameter.**

**(a)  Load a Larger Dataset**

In [29]:
import pandas as pd

# Assume the dataset is saved in a file named "train.csv" with English and French columns
# Replace with the actual path and column names based on the dataset format.
dataset_path = '/content/train.csv'
# Specify the encoding as 'latin-1' to handle the problematic characters
data = pd.read_csv(dataset_path, delimiter='\t', names=['english', 'french'], encoding='latin-1')

# Take a sample of the dataset (e.g., 10,000 pairs for faster tuning)
data = data.sample(10000, random_state=1).reset_index(drop=True)

english_texts = data['english'].values
french_texts = data['french'].values

# Tokenize, pad, and prepare sequences as done with the smaller dataset

**(b)Define a HyperModel with Keras Tuner**

In [30]:
!pip install keras-tuner -q
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    # Hyperparameters to tune
    embedding_dim = hp.Int('embedding_dim', min_value=128, max_value=512, step=64)
    lstm_units = hp.Int('lstm_units', min_value=128, max_value=512, step=64)
    learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])

    # Encoder
    encoder_inputs = Input(shape=(eng_max_length,))
    encoder_embedding = Embedding(eng_vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm, state_h, state_c = LSTM(lstm_units, return_state=True)(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(fr_max_length,))
    decoder_embedding = Embedding(fr_vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = Dense(fr_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Model compilation
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


**(c) Initialize and Run the Hyperparameter Tuning**

In [31]:
# Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of different hyperparameter combinations to try
    executions_per_trial=1,  # Number of times to train the model with each configuration
    directory='tuner_results',
    project_name='english_to_french_translation'
)

# Prepare the data for the tuner
fr_target_sequences = np.expand_dims(fr_sequences, -1)
X_train = [eng_sequences, fr_sequences]
Y_train = fr_target_sequences

# Run the hyperparameter search
tuner.search(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.2)


Reloading Tuner from tuner_results/english_to_french_translation/tuner0.json


**(d) Get the Best Model and Evaluate**

In [32]:
# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate or save the best model
best_model.evaluate(X_train, Y_train)
best_model.save('best_translation_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4762 - loss: 2.6238


**(e) Additional Improvements: Fine-Tuning and Regularization**


In [33]:
from tensorflow.keras.layers import Dropout

def build_model_with_dropout(hp):
    embedding_dim = hp.Int('embedding_dim', min_value=128, max_value=512, step=64)
    lstm_units = hp.Int('lstm_units', min_value=128, max_value=512, step=64)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)

    encoder_inputs = Input(shape=(eng_max_length,))
    encoder_embedding = Embedding(eng_vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm, state_h, state_c = LSTM(lstm_units, return_state=True)(encoder_embedding)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(fr_max_length,))
    decoder_embedding = Embedding(fr_vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_outputs = Dropout(dropout_rate)(decoder_outputs)
    decoder_dense = Dense(fr_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model
